## Logistic Regression with Gradient Descent


In [1]:
# Import all the necessary libraries
import numpy as np 
import pandas as pd 
import random

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statistics
import math
# Set the Seaborn theme


#sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report #for nice result formatting
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.decomposition import PCA


The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD).The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes.
Variables
Each attribute is a potential risk factor. There are both demographic, behavioral and medical risk factors.

In [2]:
#importing the HEART DISEASE PREDICTION dataset 
df = pd.read_csv("https://raw.githubusercontent.com/AmyrMa/INDE-577/main/data/framingham.csv")
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [4]:
df['glucose'].fillna(value=df['glucose'].mean(), inplace=True)
df['cigsPerDay'].fillna(value=0.0, inplace=True)
df['education'].fillna(value=df['education'].mean(), inplace=True)
df['totChol'].fillna(value=df['totChol'].mean(), inplace=True)
df['BPMeds'].fillna(value=df['BPMeds'].median(), inplace=True)
df['BMI'].fillna(value=df['BMI'].median(), inplace=True)
df['heartRate'].fillna(value= df['heartRate'].mean(), inplace=True)

In [5]:
y = df['TenYearCHD'].to_numpy()
X = df.drop('TenYearCHD', axis = 1).to_numpy()
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 40)
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)

X_train.shape (3178, 15) y_train.shape (3178,)
X_test.shape (1060, 15) y_test.shape (1060,)


In [6]:
def sigmoid(z) :
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b) :
    m, n = X.shape
    
    loss = 0
    for item in range(m):
        z = 0
        for feature in range(n):
            z += X[item, feature] * w[feature]
        z += b
        f_wb = sigmoid(z)
        loss += - y[item] * np.log(f_wb) - (1 - y[item]) * np.log(1 - f_wb)
    return (loss / m)

def predict(X, w, b) :
    m, n = X.shape
    p = np.zeros(m)    
    for i in range(m) :
        f_wb = sigmoid(np.dot(X[i], w) + b)
        p[i] = 1 if f_wb > 0.38 else 0
    return p


In [7]:
def compute_gradient(X, y, w, b, lambda_ = None) :
    m, n = X.shape
    dj_dw, dj_db = np.zeros(n), 0.
    
    for eg in range(m) :   
        
        f_wb = sigmoid(np.dot(X[eg], w) + b)
        err = f_wb - y[eg]
        
        for fr in range(n) :
            dj_dw[fr] += err * X[eg, fr]
        dj_db += err
            
    dj_dw /= m
    dj_db /= m
    return dj_db, dj_dw


(0.5837722104755255,
 array([-0.0295624 , -0.07551406,  0.02377325, -0.00262407, -0.02250449,
        -0.02880174, -0.01710133, -0.05959201, -0.02980786, -0.02403729,
        -0.06620869, -0.03781884, -0.01672846, -0.00679285, -0.03570121]))

In [8]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters, cost_func, gradient_func, lambda_) :   
         
    for i in range(num_iters) :
        
        dj_db, dj_dw = gradient_func(X, y, w_in, b_in, lambda_)
        w_in = w_in - alpha * dj_dw
        b_in = b_in - alpha * dj_db
        
#         For 10 iterations equally spaced of total iters, print cost.
        if i % math.ceil(num_iters / 10) == 0 or i == (num_iters - 1) :
            print(f"Iteration {i : 4}: Cost {cost_func(X, y, w_in, b_in) : 8.2f}")
            
    return w_in, b_in   

In [9]:
initial_w = np.random.rand(X.shape[1])
initial_b = -2

num_iters = 1000
alpha = 0.1
lambda_ = 0

w, b = gradient_descent(X_train, y_train, initial_w, initial_b, alpha, num_iters,compute_cost, compute_gradient, lambda_)

Iteration    0: Cost     0.56
Iteration  100: Cost     0.38
Iteration  200: Cost     0.37
Iteration  300: Cost     0.37
Iteration  400: Cost     0.37
Iteration  500: Cost     0.37
Iteration  600: Cost     0.37
Iteration  700: Cost     0.37
Iteration  800: Cost     0.37
Iteration  900: Cost     0.37
Iteration  999: Cost     0.37


In [14]:
y_pred = predict(X_test, w,b)
print("Accuracy:", accuracy_score(y_test,y_pred))
model_report = classification_report(y_test, y_pred)
print("\nClassification Report")
print(model_report)

Accuracy: 0.8386792452830188

Classification Report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       884
           1       0.57      0.12      0.20       176

    accuracy                           0.84      1060
   macro avg       0.71      0.55      0.55      1060
weighted avg       0.80      0.84      0.79      1060



## Logistic Model by python package 

In [15]:
model = LogisticRegression()

In [16]:
#Fit the model with data
model.fit(X_train, y_train)

LogisticRegression()

In [17]:
y_pred = model.predict(X_test)

In [18]:
print("Accuracy:", accuracy_score(y_test,y_pred))

Accuracy: 0.8433962264150944


In [19]:
model_report = classification_report(y_test, y_pred)
print("\nClassification Report")
print(model_report)


Classification Report
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       884
           1       0.92      0.06      0.12       176

    accuracy                           0.84      1060
   macro avg       0.88      0.53      0.52      1060
weighted avg       0.85      0.84      0.78      1060



## Conclusion
We see that our model has similar result and prediction accuracy as the python pacakge 